# Example for using the Igor reader
The igor reader allows reading data from Wavemetrics Igor Pro .ibw "binary wave" files and .pxp "packed experiment" files. These two modes of operation are mutually exclusive, i.e. you can either pass one or multiple ibw files, or one pxp file. The behavior of the reader is controlled via two config files: ``config.json`` defines the assignment to nexus class paths, whereas ``entries.yaml.entry`` defines which data to read from the source file(s). This notebook showcases several examples how to use the reader. A more complete example and template how to convert complete datasets can be found in another example notebook. These examples convert to the nexus class ``NXroot``, the most generic Nexus class.

In [ ]:
%load_ext autoreload
%autoreload 2
from pynxtools.dataconverter.convert import convert
import yaml

## Convert from ``.ibw`` Igor binary wave files

### Conversion without entry definition
The conversion from ``.ibw`` files does not require the definition of an entry file. In that case, each ibw file generates an entry with the filename (excluding ".ibw") as entry name. Axis data as well as units are read from the ibw wave information (internal wave scaling), and named ``axis0`` to ``axis3``. These are available via the ``@data`` mechanism. Wave notes are parsed and split according to a ``key=value\\n`` schema, and are available vie the ``@attrs`` mechanism.

In [ ]:
convert(
    input_file=["config_file.json", "Norm_0057.ibw", "Norm_0059.ibw"],
    reader="igor",
    nxdl="NXroot",
    output="example_ibw.nxs",
)

### Conversion with entry definition
Alternatively, an entry definition can be passed in addition, which contains a dictionary of entries, where each entry key generates an entry of that name. The dict entries are expected to be dicts as well, containing at least the key ``data`` with the filename of the respective ibw file to use as data for this entry. Additionally, ``axisN_name`` keys can be passed to rename the axis entries from the default ``axis0`` to ``axis3``. Similarly, ``axisN_units`` and ``data_units`` can be passed to overwrite respective information read from the ibw files. ibw files for which no entry has been defined will generate an entry according to their filename as before. Additionally, each entry can contain a ``metadata`` dict with additional metadata specific to this entry.

This entry dict can be passed as an object, in which case it is expected as sole and single object in the objects tuple.

In [ ]:
entry_dict = {
    "Scan57": {
        "data": "Norm_0057.ibw",
        "data_units": "counts/monitor",
        "axis0_name": "theta",
        "axis0_units": "degree",
    },
}
convert(
    input_file=["config_file.json", "Norm_0057.ibw", "Norm_0059.ibw"],
    reader="igor",
    nxdl="NXroot",
    output="example_ibw_entry.nxs",
    objects=(entry_dict,),
)

The ``entry`` dict can also be passed as yaml file with extension ``.entry``

In [ ]:
with open("Norm57.yaml.entry", "w") as f:
    yaml.dump(entry_dict, f)
convert(
    input_file=[
        "config_file.json",
        "Norm_0057.ibw",
        "Norm_0059.ibw",
        "Norm57.yaml.entry",
    ],
    reader="igor",
    nxdl="NXroot",
    output="example_ibw_entry.nxs",
)

## Conversion from Igor packed experiment files
Conversion from ``.pxp`` files requires definition of an entry dict. Here also, each key/value defines one entry to generate in the Nexus file. the ``data`` entry contains here the ibw-path to the wave to use as main data entry. Axis information can be either read from the wave scaling, or be provided via an ``axisN`` entry, pointing to a wave containing axis information. Similarly, a ``data_errors`` key can be defined, pointing to a wave containing data uncertainties.

In [ ]:
# pxp test
entry_dict = {
    "Scan57": {
        "data": "root/Norm_0057",
        "data_units": "counts/monitor",
        "axis0": "root/Raw/SPECScan_0057/QWave",
        "axis0_name": "l",
        "axis0_units": "r.l.u.",
        "metadata": {"entry_title": "Scan at 16 K"},
    },
    "Scan59": {
        "data": "root/Norm_0059",
        "data_units": "counts/monitor",
        "axis0": "root/Raw/SPECScan_0059/QWave",
        "axis0_name": "l",
        "axis0_units": "r.l.u.",
        "metadata": {"entry_title": "Scan at 18 K"},
    },
}
convert(
    input_file=["config_file.json", "Fig2a.pxp"],
    reader="igor",
    nxdl="NXroot",
    output="example_pxp.nxs",
    objects=(entry_dict,),
)

This method also can be invoked using a ``.entry`` yaml file

In [ ]:
with open("Scan57_59.yaml.entry", "w") as f:
    yaml.dump(entry_dict, f)
convert(
    input_file=["config_file.json", "Fig2a.pxp", "Scan57_59.yaml.entry"],
    reader="igor",
    nxdl="NXroot",
    output="example_pxp.nxs",
)

## Invoking from command line
Off course, the dataconverter can also be called from the command line

In [ ]:
!dataconverter --reader igor --nxdl NXroot --output example_pxp.nxs config_file.json Fig2a.pxp Scan57_59.yaml.entry